# Einfaches RAG-System Tutorial

Um das Setup möglichst einfach zu halten, werden im folgenden populäre Cloud Modelle genutzt, um den Rag-Workflow aufzubauen.

## Download der Dependencies

In [ ]:
!pip3 install --quiet numpy docling ollama matplotlib

# Dokument Laden und Parsen

Als Beispiel für ein Dokument, aus welchem Informationen extrahiert werden sollen, die nicht im statischen Wissen eines LLM enthalten sind, wird der Halbjährige Financial Report von Thales genutzt.
https://www.thalesgroup.com/sites/default/files/2025-09/Thales%20-%202025%20Half-Yearly%20Financial%20report_0_0.pdf (Zugriff 17.11.2025)

In [ ]:
from pathlib import Path
from docling.document_converter import DocumentConverter

def load_pdf_with_docling(path: str) -> str:
    converter = DocumentConverter()
    result = converter.convert(Path(path))
    doc = result.document
    markdown = doc.export_to_markdown()
    return markdown


In [ ]:
!wget -O thales_halfyear_2025.pdf "https://www.thalesgroup.com/sites/default/files/2025-09/Thales%20-%202025%20Half-Yearly%20Financial%20report_0_0.pdf"

In [ ]:
PDF_PATH = "./thales_halfyear_2025.pdf"
raw_text = load_pdf_with_docling(PDF_PATH)

len(raw_text), raw_text[:1000]


## Chunking 
In diesem Schritt wird der vom Parser gelieferte Rohtext in kleinere Textsegmente („Chunks“) zerlegt.  
Die Chunks dienen später als Grundeinheiten für das Retrieval: Das Embedding-Modell berechnet für jeden Chunk einen Vektor, und bei einer Anfrage werden diejenigen Chunks gesucht, die im Vektorraum am ähnlichsten sind.

Die Funktion `chunk_text` implementiert eine einfache, absatzbasierte Strategie:

- Zunächst wird der Text grob in Absätze aufgeteilt (Trennung über Leerzeilen).
- Absätze, deren Länge zwischen `min_chars` und `max_chars` liegt, werden direkt als eigene Chunks übernommen.
- Sehr lange Absätze werden zusätzlich in kleinere Fenster zerlegt, indem Wort für Wort ein Chunk aufgebaut wird, bis die maximale Zeichenzahl (`max_chars`) erreicht ist.
- Sehr kurze Textstücke werden verworfen, damit keine „Rauschen“-Chunks entstehen, die kaum Inhalt tragen.

In [ ]:
def chunk_text(text: str, min_chars: int = 200, max_chars: int = 800):
    """
    Einfache, absatzbasierte Chunking-Funktion für Markdown-Text.

    - Der Eingabetext wird zunächst grob in Absätze zerlegt (Trennung über Leerzeilen).
    - Absätze, deren Länge zwischen min_chars und max_chars liegt, werden direkt als Chunks übernommen.
    - Absätze, die länger als max_chars sind, werden in kleinere Fenster (Chunks) zerlegt.
    """
    raw_paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]

    chunks = []

    for para in raw_paragraphs:
        if len(para) <= max_chars:
            if len(para) >= min_chars:
                chunks.append(para)
        else:
            words = para.split()
            current_words = []

            for w in words:
                current_words.append(w)
                # Sobald der aktuelle Fenster-Text max_chars erreicht, wird ein Chunk abgeschlossen
                if len(" ".join(current_words)) >= max_chars:
                    chunk = " ".join(current_words)
                    chunks.append(chunk)
                    current_words = []

            # Rest am Ende des Absatzes, der keinen eigenen vollen Chunk mehr bildet
            if current_words:
                tail = " ".join(current_words)
                # Nur übernehmen, wenn er nicht zu kurz ist oder bisher noch gar kein Chunk existiert
                # (damit zumindest ein Chunk erzeugt wird)
                if len(tail) >= min_chars or not chunks:
                    chunks.append(tail)

    return chunks


chunk_texts = chunk_text(raw_text)
chunks = [{"id": i, "text": t} for i, t in enumerate(chunk_texts)]
print(f"Anzahl Chunks: {len(chunks)}")


Für das Tutorial werden lokal betriebene Modelle verwendet: embeddinggemma:300m für Embeddings und gemma3:27b als LLM. Beide Modelle werden über Ollama geladen und ausgeführt, sodass keine externen Cloud-Dienste benötigt werden

Alternativ zu gemma3 kann jedes andere Ollama unterstütze Model geladen werden. Siehe: https://ollama.com/search

In [ ]:
import ollama

EMBEDDING_MODEL = "embeddinggemma:300m"
CHAT_MODEL = "gemma3:27b"

try:
    _ = ollama.list()
except Exception as e:
    raise RuntimeError("Ollama scheint nicht zu laufen. Bitte Ollama-App oder 'ollama serve' starten.") from e

## Embeddings der Chunks mit `embeddinggemma:300m`

- In diesem Schritt werden alle zuvor erzeugten Text-Chunks in Vektoren (Embeddings) überführt.
- Dieser Index bildet die Grundlage für das spätere Retrieval (Top-k-Suche nach relevanten Chunks zur Nutzeranfrage).


In [ ]:
import numpy as np

def embed_texts(texts: list[str]) -> np.ndarray:
    """
    Erzeugt Embeddings lokal über Ollama mit embeddinggemma:300m.
    Erwartet eine Liste von Texten und liefert ein numpy-Array (N x d) zurück.
    """
    response = ollama.embed(
        model=EMBEDDING_MODEL,
        input=texts,
    )
    vectors = response["embeddings"]
    return np.array(vectors, dtype="float32")

chunk_texts = [c["text"] for c in chunks]
chunk_embeddings = embed_texts(chunk_texts)

print("Anzahl Chunks:", len(chunks))
print("Shape der Embeddings:", chunk_embeddings.shape)

index = {
    "chunks": chunks,
    "embeddings": chunk_embeddings,
}

### Retrieval über Cosine Similarity

Die Funktion `cosine_similarity` berechnet die Ähnlichkeit zwischen der eingebetteten Nutzeranfrage und allen Chunk-Embeddings.  
`retrieve` nutzt diese Ähnlichkeiten, sortiert sie absteigend und gibt die Top-k Chunks mit höchstem Score als Retrieval-Ergebnis zurück.


In [ ]:
def cosine_similarity(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Cosine Similarity zwischen einem Vektor a (1 x d) und vielen Vektoren b (N x d).
    """
    a_norm = a / np.linalg.norm(a, axis=-1, keepdims=True)
    b_norm = b / np.linalg.norm(b, axis=-1, keepdims=True)
    return np.dot(a_norm, b_norm.T)


def retrieve(query: str, index: dict, k: int = 5) -> list[dict]:
    """
    Embed der Query, Cosine Similarity, Top-k-Chunks zurückgeben.
    """
    q_emb = embed_texts([query])
    sims = cosine_similarity(q_emb, index["embeddings"])[0]

    top_k_idx = np.argsort(sims)[-k:][::-1]

    results = []
    for i in top_k_idx:
        results.append({
            "score": float(sims[i]),
            "id": index["chunks"][i]["id"],
            "text": index["chunks"][i]["text"],
        })
    return results


### Prompt-Konstruktion für das LLM

`build_rag_prompt` kombiniert die Nutzerfrage mit den Top-k-Retrieval-Chunks zu einem einzigen Prompt.  
Der Kontext wird dabei explizit eingefügt, und das Modell wird angewiesen, seine Antwort ausschließlich auf diese Textpassagen zu stützen.


In [ ]:
def build_rag_prompt(question: str, retrieved_chunks: list[dict]) -> str:
    """
    Baut einen RAG-Prompt aus Frage + Top-k-Chunks.
    """
    context_parts = []
    for r in retrieved_chunks:
        context_parts.append(f"[Chunk {r['id']}] {r['text']}")
    context = "\n\n".join(context_parts)

    prompt = f"""
Du beantwortest Fragen ausschließlich auf Basis des bereitgestellten Kontextes.
Wenn eine Information im Kontext nicht enthalten ist, sage ausdrücklich, dass sie nicht vorliegt.
Erfinde keine Fakten.

Kontext:
{context}

Frage:
{question}

Antwort (knapp, sachlich, auf Deutsch):
""".strip()

    return prompt


### End-to-End-RAG-Aufruf

`answer_question` führt den gesamten RAG-Durchlauf aus:  
Zuerst werden per `retrieve` die Top-k-Chunks gefunden, daraus mit `build_rag_prompt` ein Prompt gebaut und dieser anschließend an `gemma3:27b` über Ollama geschickt. Die Funktion gibt die Modellantwort sowie die verwendeten Chunks zurück.


In [ ]:
def answer_question(question: str, index: dict, k: int = 5) -> tuple[str, list[dict]]:
    """
    Vollständiger RAG-Durchlauf:
    - Retrieval der Top-k-Chunks
    - Prompt-Konstruktion
    - Antwort von gemma3:27b über Ollama

    Rückgabe: (Antworttext, verwendete Chunks)
    """
    retrieved = retrieve(question, index, k=k)
    prompt = build_rag_prompt(question, retrieved)

    response = ollama.chat(
        model=CHAT_MODEL,
        messages=[
            {
                "role": "system",
                "content": "Du bist ein hilfreicher Assistent für Fragen basierend auf den dir vorliegenden Informationen."
            },
            {
                "role": "user",
                "content": prompt
            },
        ],
    )

    answer = response["message"]["content"]
    return answer, retrieved


### Beispielanfrage an das RAG-System

Zum Abschluss wird eine Beispielfrage an das RAG-System gestellt.  
Die Funktion `answer_question` liefert dabei sowohl die Antwort von `gemma3:27b` als auch die dafür herangezogenen Chunks, die zur Kontrolle mit Score und Chunk-ID ausgegeben werden.


In [ ]:
question = "Welche Veränderungen gab es in den Vorstandsgehältern oder der Vergütungsstruktur zwischen H1 2024 und H1 2025?"
answer, retrieved = answer_question(question, index, k=4)

print("Frage:\n", question)
print("\nVerwendete Chunks:\n")
for r in retrieved:
    print(f"- (Score {r['score']:.3f}, Chunk-ID {r['id']}):\n{r['text'][:300]}...\n")

print("Antwort von gemma3:27b:\n")
print(answer)


## QA-Benchmark

Für den Test wird eine KI als Judge verwendet, um zu bewerten, ob die Modell antworten inhaltlich denen im Dataset entspricht.
Dafür muss im folgenden ein API Key für OpenAI angegeben werden, wofür ein Account benötigt wird.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "key"


## Ausführen des Benchmarks

Im Anschluss wird eine Grafik erzeugt, welche die korrekten Antwort (in %) pro Kategorie visualisiert.

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()

with open("qa_benchmark.json", "r", encoding="utf-8") as f:
    qa_dataset = json.load(f)

print("Loaded", len(qa_dataset), "Benchmark Items")

class JudgeScore(BaseModel):
    score: float  # 0 bis 1


def judge_answer(question: str, model_answer: str, gold_answer: str) -> float:
    """
    Bewertet die Modellantwort über GPT mithilfe eines
    strikt erzwungenen JSON-Ausgabeformats (Pydantic).
    """
    prompt = f"""
Bewerte die Modellantwort im Vergleich zur Goldstandard-Antwort inhaltlich.

Kriterien:
- Korrektheit
- Vollständigkeit
- Faktentreue

Gib eine einzige Zahl zwischen 0 und 1 zurück:
1.0 = vollständig korrekt
0.5 = teilweise korrekt
0.0 = falsch oder halluziniert

Frage:
{question}

Goldstandard:
{gold_answer}

Modellantwort:
{model_answer}
"""

    response = client.responses.parse(
        model="gpt-5.1",
        input=[{"role": "user", "content": prompt}],
        text_format=JudgeScore,
        max_output_tokens=20
    )

    return float(response.output_parsed.score)

results = []

for item in qa_dataset:
    q = item["question"]
    gold = item["answer"]

    model_answer, used_chunks = answer_question(q, index, k=4)
    score = judge_answer(q, model_answer, gold)

    results.append({
        "id": item["id"],
        "section": item["section"],
        "type": item["type"],
        "question": q,
        "gold_answer": gold,
        "model_answer": model_answer,
        "score": score,
        "used_chunks": [c["id"] for c in used_chunks],
    })

    print(f"{item['id']} — Score: {score:.3f}")


df = pd.DataFrame(results)
df.to_csv("rag_eval_results.csv", index=False)

with open("rag_eval_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("\nEvaluation finished.")
print("Saved rag_eval_results.csv + rag_eval_results.json")


threshold = 0.85
df["correct"] = df["score"] >= threshold

type_stats = df.groupby("type")["correct"].mean() * 100
type_stats = type_stats.sort_values(ascending=False)

plt.figure(figsize=(10, 5))
bars = plt.bar(type_stats.index, type_stats.values, color="#4a90e2")

for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height + 1,
        f"{height:.1f}%",
        ha="center",
        va="bottom",
        fontsize=10
    )

plt.title("RAG-Answer-Quality pro Typ (in % korrekt beantwortet)", fontsize=14)
plt.ylabel("Korrekt beantwortet (%)", fontsize=12)
plt.ylim(0, 105)
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()